In [ ]:
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
from lora_diffusion import tune_lora_scale, patch_pipe
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
prompt = "<s1>"
model_id = "stabilityai/stable-diffusion-2-1-base"
tensors_path = "/Downloads/sex/step_100.safetensors"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(
    "cuda"
)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
torch.manual_seed(0)
image = pipe(prompt, num_inference_steps=25, guidance_scale=9).images[0]
image

In [ ]:
def display_grid(images, rows, cols, title):
    fig, axs = plt.subplots(rows, cols, figsize=(15, 6))
    for i, ax in enumerate(axs.flat):
        ax.imshow(images[i])
        ax.axis('off')
    fig.suptitle(title)
    plt.show()
    
def test_gen():
    torch.manual_seed(0)
    images = [pipe(prompt, num_inference_steps=25, guidance_scale=9).images[0] for _ in range(2)]
    display_grid(images, 1, 2, prompt)

In [ ]:
patch_pipe(
    pipe,
    tensors_path,
    patch_text=True,
    patch_ti=True,
    patch_unet=True,
)

In [ ]:
tune_lora_scale(pipe.unet, 1)
tune_lora_scale(pipe.text_encoder, 1)
test_gen()

In [ ]:
tune_lora_scale(pipe.unet, 0.8)
tune_lora_scale(pipe.text_encoder, 0.8)
test_gen()